# Dataset Inspector for: Learning Not To Learn, CVPR 2019
Paper: https://arxiv.org/abs/1812.10352  
Download dataset from https://drive.google.com/file/d/1NSv4RCSHjcHois3dXjYw_PaLIoVlLgXu/view?usp=sharing
### Requirements
- Python 3.6
- NumPy

## Load from the .npy files

In [41]:
import os
from tkinter import Tk, filedialog
import numpy as np

root = Tk()
filepath = filedialog.askopenfilename(title='Select file', filetypes = (("numpy files","*.npy"),("all files","*.*")))
filepath = os.path.normpath(filepath)
root.withdraw()

dataset = np.load(filepath, allow_pickle=True, encoding='latin1')
dataset = dataset.item()
print(dataset.keys())

dict_keys(['test_image', 'test_label', 'test_gray', 'train_label', 'train_image'])


## Take a look of few samples

In [42]:
for k, v in dataset.items():
    print('{}: {}'.format(k, v.shape))

test_image: (10000, 28, 28, 3)
test_label: (10000,)
test_gray: (10000, 28, 28)
train_label: (60000,)
train_image: (60000, 28, 28, 3)


In [57]:
from PIL import Image as img
from IPython.display import display 

for x in dataset['train_image'][:10]:
    display(img.fromarray(x))
    x = np.transpose(x, (2,0,1))
    print(x[:,0].shape)

(3, 28)


(3, 28)


(3, 28)


(3, 28)


(3, 28)


(3, 28)


(3, 28)


(3, 28)


(3, 28)


(3, 28)


In [44]:
for x in dataset['test_gray'][:10]:
    display(img.fromarray(x))

In [45]:
dataset['train_label']

array([5, 0, 4, ..., 5, 6, 8], dtype=int8)

## Extract bias labels from color information

In [46]:
train_bias = []
for x in dataset['train_image']:
    r = x[..., 0].max()
    g = x[..., 1].max()
    b = x[..., 2].max()
    train_bias.append([r,g,b])
train_bias = np.array(train_bias)

dataset.update({'train_bias': train_bias})